# Check `FuturesTrade` tracking code

* Compare trades between two subgraphs
* Add bytes decoding
* Ensure upgraded subgraph has tracking codes set


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.express as px

load_dotenv()


True

In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
OLD_SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-perps'

SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [3]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)
convertBytes = lambda x: bytearray.fromhex(x[2:]).decode().replace('\x00', '')

def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [4]:
old_trade_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
    }  
}
""")


trade_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
      trackingCode
    }  
}
""")


### Run a query

In [5]:
trade_params = {
    'last_id': ''
}

trade_decimal_cols = [
    'price',
    'margin',
    'feesPaid',
    'pnl',
    'size',
    'positionSize',
]

trade_bytes_cols = ['trackingCode']

df_trade = await run_recursive_query(trade_query, trade_params, 'futuresTrades')
df_trade = df_trade.replace({None: np.nan})
df_trade = clean_df(df_trade, decimal_cols=trade_decimal_cols, bytes_cols=trade_bytes_cols)


In [6]:
df_trade

,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,trackingCode
0,0x0003d5401b4c7a8a1feb9e8ea750336e6df42b5cd3ff...,0x50d36f27872f637ec9ecb4eedff300df44a608ab,1673640747,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1420.509945340577467893,83.803545657223511638,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x26b,3.128026947594952567,0,-1.5882,0,True,KWENTA
1,0x00232eaefdd653149d9ae21764a1f412d548a5b6dd4d...,0xa110d783a5b6b0b2a2c867094364e8f9752d5b93,1675964339,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1613.322643212031584915,19212.662872131648312194,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x5f5,331.155086968312641252,0,-204.0231,0,True,KWENTA
2,0x0024ea62644b2054a4e2acc5e37b6291fa2973117a33...,0xadc6461555c94217d14351ded5f5583634c08191,1673742652,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1542.283461883403867843,398.000000000000080497,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x2b1,4.698996058295956768,0,-3.5,-3.5,False,decentrex
3,0x00268abb5859495ac5ed1519f64ae84b1f7c50eef1d7...,0xc9c67603d3107ad53d1816d3096b5b36b52cdedc,1675342588,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1675.63495283188297317,43.758239544222270206,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x4d1,2.000004212759417786,0,0.000005028254406685,-4.14103137E-10,False,polynomial
4,0x00277c831e2e2e46b79037471586b02ad80bea2bac4f...,0x83106ddcac5d119a3d0f551e06239e579299b7c4,1676309220,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1475.133501091447750939,83.506999804202433774,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x565,2.033672626866002274,0,0.022826833531397652,-0.114378935336416095,False,DHEDGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7072,0xffd64e62c995b0cde44238bfa23d9bf789f5a1f0e102...,0xf0753b791d9afa14e4f0963c2f9b603282d593fd,1673542151,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1385.055469509376174366,108.221843840186870326,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x23d,2.364182547749186308,0,1.969859803856687971,1.969859803856687971,False,KWENTA
7073,0xffd7c7fe1e4e726b4e93910453d6c07343dc28045f7e...,0x1bd477fc34b96ab561156f2438b8f9a18fcb5673,1676274577,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1522.77873592749243167,251.247711634623038918,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x723,8.335012093830431712,0,4.160165849683939389,4.160165849683939389,False,KWENTA
7074,0xffe545b86e9a47ba061ddbe7f20d4729169f347009c1...,0x0c8aa570a1dfeee5258f3c13e2e967da24bbb505,1673262644,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1322.79752629640388717,52.755917631435047066,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x190,1.137240243353251903,0,0.2075,0,True,KWENTA
7075,0xffe55e5a83b9be19c56cbb873966c8fa586b66c24422...,0x182a36ad47ea21d96d864080b7baceda7ce44019,1676398634,0x73455448000000000000000000000000000000000000...,0x73455448504552500000000000000000000000000000...,DelayedOffchain,1550.020923468874649187,2018.301296096519858795,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x7aa,4.445700514095363865,0,3.1557,15.9371,False,KWENTA


## Get the old data and compare

In [7]:
old_trade_params = {
    'last_id': ''
}

df_old_trade = await run_recursive_query(old_trade_query, old_trade_params, 'futuresTrades', endpoint=OLD_SUBGRAPH_ENDPOINT)
df_old_trade = df_old_trade.replace({None: np.nan})
df_old_trade = clean_df(df_old_trade, decimal_cols=trade_decimal_cols)


In [19]:
df_trade.shape

(7077, 17)

In [20]:
df_trade['trackingCode'].value_counts()

KWENTA        5613
decentrex      616
polynomial     565
               264
DHEDGE          19
Name: trackingCode, dtype: int64

In [21]:
df_old_trade.shape

(7077, 14)

In [39]:
df_trade_compare = df_trade.drop('trackingCode', axis=1).sort_values('id')

In [42]:
df_trade_compare['feeDiff'] = df_trade_compare['feesPaid'][(df_trade_compare['feesPaid'] != df_old_trade['feesPaid'])] - df_trade_compare['feesPaid'][(df_trade_compare['feesPaid'] != df_old_trade['feesPaid'])]


## Plot some of the tracking code data

In [43]:
df_trade['size_usd'] = df_trade['size'] * df_trade['price']
df_trade['day'] = df_trade['timestamp'].apply(lambda ts: datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d'))


In [44]:
df_fees = df_trade.groupby(['day', 'trackingCode'])['feesPaid'].sum().reset_index()
df_fees['feesPaid'] = df_fees['feesPaid'].astype(np.float64)

In [45]:
fig = px.bar(
    df_fees,
    x='day',
    y='feesPaid',
    color='trackingCode',
)
fig.show()


In [46]:
df_fees.groupby('day')['feesPaid'].sum().tail(25)

day
2023-01-22     4120.326487
2023-01-23      682.491159
2023-01-24     7515.175319
2023-01-25     3907.575470
2023-01-26     2309.134444
2023-01-27     1099.283485
2023-01-28      626.411977
2023-01-29     3431.633992
2023-01-30     8669.410773
2023-01-31     2334.078679
2023-02-01     7483.179520
2023-02-02     4336.623824
2023-02-03     2925.378704
2023-02-04      478.471722
2023-02-05     1912.904856
2023-02-06     5582.977081
2023-02-07     3198.930321
2023-02-08     4419.030338
2023-02-09    26076.789211
2023-02-10    11349.227274
2023-02-11     8862.377909
2023-02-12     9816.440017
2023-02-13    19730.460676
2023-02-14    10273.127024
2023-02-15    14134.200929
Name: feesPaid, dtype: float64